# Abstract

Our goal is to design a single response dialogue system based on a series of back and forth dialogues (utterances).  We intend to show that a hierarchical encoder-decoder neural network’s (HRED) framework, which can retain context at both the utterance level and the conversation level, will produce more reasonable responses than an ngram model. Performance of each model will be evaluated with perplexity scores.  

# Introduction

The exploration into an HRED model can provide the building block for a more robust chatbot system.  Chatbots have recently become a popular area of research with the rise of NLP and Deep Learning frameworks.  Significant time and money has been spent into developing chatbots with the ultimate goal of creating chatbots that can produce responses and interactions that are indistinguishable from human to human interactions.  This goal has yet to be achieved, however research in the area has made great progress over the past few years.

There are many challenges with developing chatbots that can produce coherent responses, but one of the most difficult is keeping track of linguistic and physical context during a conversation.  This is made even more complex for open domain generative models which may need to track conversations over many turns and must have the adaptability to respond to unfamiliar words and phrases.  Although we will not be implementing a fully functional chatbot, our focus will be to implement the basic idea behind a chatbot with single sentence responses given an open domain context.  


# Background

An end to end dialogue system utilizing a HRED framework was proposed by Sordoni [1].  The HRED model described by Sordoni [1] makes the assumption that a dialogue can be seen as a sequence of utterances which, in turn, are sequences of tokens. The hierarchical model captures context through two levels of RNNs, one at the word level and one at the utterance level. This information is encoded to vector representations and passed to a decoder RNN which outputs an utterance based on the probability distribution of tokens to create a response given the context from the provided dialogue. The response ends when the decoder RNN output an end of sentence token.  

One of the main benefits of this model is that it can provide context awareness that cannot be captured in more traditional count based model such as ngram model.  At the input level of the HRED, words are parameterized using a co-occurrence matrix which results in a vector representation of all the words in the vocabulary.  The word embeddings enable the model to learn over a greater context windows compared to an n-gram model.  Using standard n-grams to compute joint probabilities over dialogues, e.g. computing probability tables for each token given the n preceding tokens, suffers from the curse of dimensionality and is intractable for any realistic vocabulary size [1].  



# Methods
**Dataset**

We will be using the movie dialogues corpus (Movie-DiC) as our main dataset described by Banchs [3].  This is a corpus scraped from the internet movie script data collection.  The dialogue corpus contains 132,229 dialogues containing a total of 764,146 turns which have been extracted from 753 movies.  The Movie-DiC  was created with the intent to provide a wide range of situational, emotional, and stylistic dialogue contexts which should provide a strong training foundation for an open domain conversational agent.  The dataset has been preprocessed to reduce sparsity issues.  This includes replacing name tokens with `<person>` and number tokens with `<number>`.  In addition all words have been made lowercase and all but the 10,000 most common tokens have been replaced with a `<unk>` token.  Like many corpus the tokens follows a power law distribution where we have a steep drop off for the most common tokens and a extremely long tail. 

**Hierarchical Recurrent Encoder-Decoder Model**

We propose using the architecture of a hierarchical recurrent encoder-decoder framework proposed by Sordoni [2] used originally to build a context aware query suggestion system.  The first part of the model is a encoder Recurrent Neural Network (RNN).  The encoder RNN will take in a vector representations of words (embeddings) from an initial utterance in a dialogue sequence.  Embeddings project words that share similarities in meaning and context within sentences or phrases into similar areas of vector space.  The encoder RNN maintains the order of embeddings seen in the initial utterance by updating the hidden state and outputs an encoding which provides the context retained from the initial utterance for the second part of the model, the decoder RNN.  The decoder RNN is initialized with the encoding and generates an utterance based on the probability of tokens in a given corpus and the response utterance is generated and completed when the model outputs an end of sentence token.


**Figure 1 : Two levels of RNN [Sordoni [1] ]**
![HRED](HRED.png)

**Gated Recurrent Unit**

Within the RNN cell we will make use of a Gated Recurrent Unit (GRU).  GRUs will allow us to store context for longer term dependencies and have demonstrated to achieve better performance than simpler parameterizations at an affordable computational cost [Sordoni et al [1] ].  Figure 2 below shows the equations for a GRU.  There are two gates in a GRU, a reset gate $ r $ and an update gate $ z $.  At a high level the reset gate combines the current context with the previous context and the update gate determines how much of the previous context to retain.  $ h $ and $ s_{t} $ represent retained or discarded memory in the GRU. 

**Figure 2: GRU Equations**

- **Reset Gate** : $ z = \sigma(x_{t}U^z + s_{t-1}W^z) $
- **Update Gate** : $ r = \sigma(x_{t}U^r + s_{t-1}W^r) $
- **Candidate Update** : $ h = tanh(x_{t}U^h + (s_{t-1}*r)W^h) $
- **Final Update** : $ s_{t} = (1 - z) * h + z * s_{t-1} $

# Results and Discussion

**N-Gram Model**

We used an n-gram Model as our baseline model where we tried both Add-k Smoothing and Kneser-Ney Smoothing.  For the model we computed the perplexity of the training and test data.  The input to the model is a two turn conversation (two utterances) and the model predicts a response, the third utterance in our dataset.

For the the n-gram model, we chose n=4 . This is because each sentence ends with . & `< s >` . Thus, at a minimum we need to look at the last 4 words to obtain a better context to predict the next word. If we choose a Trigram model, we will always predict the same sequence of words since the context for every predicted sentence will be the same.  We have taken two approaches for predicting the next word, the first is where we output the word with the highest probability of occurrence, the second is utilizing Beam Search.  

**Results**

With Add-K Smoothing, we see that the sentences are not that meaningful (Table 1). We run into sparsity issues with the 4-gram model.  Once the context is unknown, all the words have an equal probability of being the next word, which results in a random choice of words in our vocabulary.  The issues with the Add-k model are reflected in the high perplexity score of 990 achieved on our test data set. 

The results from a Kneser Ney model are shown in Table 2.  It is clear that the sentences produced by this model produce more fluent results and the perplextity score of 104 on the test data support this initial evaluation.  As expected a Kneser Ney model performs better than Add-k because it is able to address some of the sparsity issues we encountered prior.  Specifically we fall back to lower order n-grams to provide more robust context information for the model.

Initially we approached our baseline prediction models with max probability prediction, however Beam Search proved to be more effective.  This is because beam search evaluates multiple sequence paths increasing the likelihood we find a more optimal path when compared with max probability.  In the case of the baseline calculation we pruned all but the top two results.   With max probability, the same sentence has the highest probability of occurring for every input, making it less effective then beam search.

**Table 1: Sample Sentences (Add-K with Beam Search)**

| Input Sequence | Expected Response | Predicted Response      
|----------|----------|
|accident . you got to be more careful . hands is important . let me show you something . when i was a kid , now i don ' t know if you can still see it , but i gashed my fingers in a lawnmower .| `<person>` ' t let me die , `<person>` . i don ' t want to die . | miranda gross sabot emerge rsvp hemingway goin rapture molecule partition
| stop it - he ' s your landlord - you can ' t be picking fights with him . `<person>` guineas .| you don ' t give him enough credit chas . `<continued_utterance>` we better go .| sponsored saks wolfi anarchy upon summon mid-life agreeing grab-ass hartford
|it ' s got iron in it . oh . is that good or bad ?|what are you doing on the reservation ?|desoto -c eyars irritability booted bound prophetic wh-why mel lobster

**Table 2: Sample Sentences (Kneser-Ney with Beam Search)**

| Input Sequence | Expected Response | Predicted Response      
|----------|----------|
|would i even have told you they were gone if i had anything to hide ? but why didn ' t you kennel that dog right away ?|aw , `<person>` , i can ' t take it anymore . i ' m gon na pack my bags and go back to my own place|`<s>` no , no . `<s>` you ' re a
|this is all we have . some weather we ' re having .|you don ' t give him enough credit chas . `<continued_utterance>` we better go .|must be `<person>` ' first time out of ten ,
|you hear that ? i hear the humming . `<person>` electricity .|what are you doing on the reservation ?|`<s>` what ' s the matter with him . `<s>`

**Table 3: Baseline Model Scores**

|Model|Perplexity Score|
|--|:--:|
|Add-K with Beam Search |990.43|
|Kneser-Ney with Beam Search |103.55|

**Hierarchical Recurrent Encoder Decoder Model**

# References

[1] Iulian V. Serban , Alessandro Sordoni, Yoshua Bengio, Aaron Courville and Joelle Pineau  2016. Building End-To-End Dialogue Systems Using Generative Hierarchical Neural Network Models.

[2] Alessandro Sordonif , Yoshua Bengiof , Hossein Vahabig , Christina Liomah , Jakob G. Simonsenh , Jian-Yun Nief 2015. A Hierarchical Recurrent Encoder-Decoder for Generative Context-Aware Query Suggestion.

[3] Banchs, R. E. 2012. Movie-DiC: A movie dialogue corpus for research and development. In Proceedings of the 50th Annual Meeting of the Association for Computational Linguistics, 203–207.